In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import geopandas as gpd
import h3pandas 
sys.path.append('../../../reproducible_figures/scripts/')
import rep_fig_vis as rfv
sys.path.append('../../scripts/')
import create_dataset_utils as cdu
sys.path.append('../../../cnn-land-cover/scripts/')
import land_cover_analysis as lca 
import land_cover_visualisation as lcv

sys.path.append('../../scripts/')
import create_dataset_utils as cdu
import paired_embeddings_models as pem
import data_vis_utils as dvu
import shapely
from tqdm import tqdm, tqdm_notebook
import os
import pandas as pd

Font size is set to 10


In [15]:
tmp = pem.load_stats(timestamp='2024-03-02-0232')

Loaded stats from PECL-ImEn_2024-03-02-0232_vnum-243_stats.pkl at /Users/t.vanderplas/models/PECL/stats/


In [246]:
# tmp_df, tmp_details = dvu.create_df_list_timestamps(list_ts=[
#     '2024-03-01-0055', 
#     '2024-03-01-0109', 
#     '2024-03-01-0125', 
#     '2024-03-01-0146', 
#     '2024-03-01-0205', 
#     '2024-03-01-0222',
#     ])

tmp_df, tmp_details = dvu.create_df_list_timestamps(list_ts=dvu.get_list_timestamps_from_vnums(
    list_vnums=np.arange(199, 217)
))

Font size is set to 10
Loaded stats from PECL-ImEn_2024-02-29-2335_vnum-199_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-02-29-2346_vnum-200_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-02-29-2358_vnum-201_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0010_vnum-202_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0025_vnum-203_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0035_vnum-204_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0055_vnum-205_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0109_vnum-206_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0125_vnum-207_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024

In [251]:
df_print = dvu.create_printable_table(df=tmp_df, hparams_use=tmp_details[1], metrics_use=tmp_details[2],
                           metric_optimise=tmp_details[3], # hparam_show='alpha_ratio_loss',
                           save_table=True, filename='tab_test.tex', highlight_best_row=True)

Hyperparameter alpha_ratio_loss has 2 unique values
Hyperparameter seed_used has 3 unique values
Hyperparameter batch_size_used has 3 unique values
No hyperparameters to show specified, using ['alpha_ratio_loss', 'batch_size_used']


In [249]:
df_print[0]
# df_print[0]

alpha_ratio_loss batch_size_used    Top-10               Top-5            \
                                        mean       sem      mean       sem   
0              0.0              32  0.706468  0.003139  0.624378  0.006468   
1              0.0              64  0.706841  0.001385  0.624129  0.007735   
2              0.0             128  0.707463  0.002428  0.623881  0.007363   
3              0.1              32  0.703358  0.002484  0.628358  0.003756   
4              0.1              64  0.709080  0.002125  0.629602  0.005738   
5              0.1             128  0.707338  0.001558  0.626119  0.006934   

        MSE            
       mean       sem  
0  0.012006  0.000240  
1  0.012137  0.000319  
2  0.012064  0.000306  
3  0.012225  0.000421  
4  0.011985  0.000338  
5  0.012188  0.000365

In [250]:
df_print[1]

,$\alpha$,Batch size,Top-10 [\%],Top-5 [\%],MSE [1e+02]
0,0.0,32,70.6 $\pm$ 0.3,62.4 $\pm$ 0.6,1.20 $\pm$ 0.02
1,0.0,64,70.7 $\pm$ 0.1,62.4 $\pm$ 0.8,1.21 $\pm$ 0.03
2,0.0,128,70.7 $\pm$ 0.2,62.4 $\pm$ 0.7,1.21 $\pm$ 0.03
3,0.1,32,70.3 $\pm$ 0.2,62.8 $\pm$ 0.4,1.22 $\pm$ 0.04
4,0.1,64,\textbf{70.9 $\pm$ 0.2},\textbf{63.0 $\pm$ 0.6},\textbf{1.20 $\pm$ 0.03}
5,0.1,128,70.7 $\pm$ 0.2,62.6 $\pm$ 0.7,1.22 $\pm$ 0.04


In [245]:
for c in df_print[1].columns:
    print(df_print[1][c].dtype)

object
int64
object
object
object


In [241]:
tmp = [1, 2]
tmp.remove(2)
tmp

[1]